In [ ]:
# additional colab/kaggle setup
import sys
import os

def download_data():
    !git clone https://github.com/tobihol/aapor-finetuning.git
    %mv aapor-finetuning/data/ .
    %rm -rf aapor-finetuning/

    return


def is_running_in_kaggle():
    return 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

def is_running_in_colab():
    return "google.colab" in sys.modules

if is_running_in_colab() or is_running_in_kaggle():
    print("Running on Colab/Kaggle")
    download_data()
else:
    print("Not running in Colab/Kaggle")

# Exercise 1: Model Selection

Think about your use case an pick a model. 

**Questions to consider:**

- Do you want to run it on your own hardware?
- Do you have funds to use for a proprietary model?
- Do you have strong programming skills?

**Example use-case:**

- Predicting vote choice in the US presidential election based on ANES survey answer.

**Resources**

- The popular benchmark **Chatbot Arena** _(Chiang et al., 2024)_  provides a live updated elo score based on crowd-sourced pairwise comparisons of their responses.
    - Trade-off Plot: https://lmarena.ai/price
- The **Stanford HELM Benchmark** evaluates language models on over 50 tasks, including understanding and reasoning. It provides insights into model performance in terms of accuracy, fairness, and robustness.
    - https://crfm.stanford.edu/helm/
- The **Artificial Analysis** platform provides independent analysis of AI models and API providers. The main metrics are 'intelligence', speed, and price. 
    - https://artificialanalysis.ai/
- The **Open LLM Leaderboard** on Hugging Face runs multiple well established benchmarks on for open-source models hosted on the site. It is most useful to compare smaller open-source LLMs that are not featured on Chatbot Arena or compare different fine-tunes of models. 
    - https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard/

**Notes**

Feel free to make notes here, etc.

# Exercise 2: Excel to JSON

What would the corresponding JSON entry look like?

|   Age | Gender   | State     | Vote Choice   |
|------:|:---------|:----------|:--------------|
|    27 | female   | Louisiana | Trump         |
|    45 | male     | nan       | Clinton       |
|   nan | female   | Ohio      | Non-voter     |


**Example**

Excel
|   Age | Gender | Education | Occupation |
|------:|:-------|:----------|:-----------|
|    30 | male   | bachelor  | accountant |
|    24 | female | master    | engineer   |

JSON
```json
{
    "participants": [
        {
            "age": 30,
            "gender": "male",
            "education": "bachelor",
            "occupation": "accountant"
        },
        {
            "age": 24,
            "gender": "female",
            "education": "master",
            "occupation": "engineer"
        }
    ]
}
```

**Extra Question:**
Are there different ways to construct the JSON?

In [22]:
# type up your answer here

json_data = {
    "participants": [
        # TODO
    ]
}

In [ ]:
# In practice this would be done automatically,
# here is are two different examples of how to do so

import pandas as pd

data = pd.read_excel("data/json_task.xlsx", index_col=0)

print("--- Records Orientation ---")
print(data.to_json(orient="records"))
print()

print("--- Columns Orientation ---")
print(data.to_json(orient="columns"))
print()

# Exercise 3: JSON Prompt

Build a prompt for your task.

**Questions to consider:**
- What should the output look like?
- Should the model act as an expert in a certain field? Who does it substitute?


**Example use-case:**
- Predicting vote choice in the US presidential election based on ANES survey answer

**Example Conversion**

SURVEY DATA
```json
{
    "participants": [
        {
            "age": 30,
            "gender": "male",
            "education": "bachelor",
            "occupation": "accountant"
        },
    ]
}
```

PROMPT
```json
{
    "prompt": [
        {
            "role": "system",
            "content": "You are a survey participant. Reply to the user's question with a short answer."
        },
        {
            "role": "user",
            "content": "What is your age?"
        },
        {
            "role": "assistant",
            "content": "I am 30 years old."
        },
        {
            "role": "user",
            "content": "What is your gender?"
        },
        {
            "role": "assistant",
            "content": "I am a male."
        },
        {
            "role": "user",
            "content": "What is your occupation?"
        },
        {
            "role": "assistant",
            "content": "I am an accountant."
        }
    ]
}
```

In [30]:
# type up your answer here

json_prompt = {
    "prompt": [
        # TODO
    ]
}

We can test our prompt with a small open-source model.

In [28]:
from transformers import pipeline

# id of a model hosted on Hugging Face
model_id = "unsloth/Llama-3.2-1B-Instruct"

# Create a pipeline for an instruct model using the json_prompt as input
instruct_pipeline = pipeline("text-generation", model=model_id, device_map="auto")

In [ ]:
# Generate a response using the instruct model pipeline
responses = instruct_pipeline(json_prompt["prompt"], num_return_sequences=10)

# Print the generated response
print("10 different generated responses:")
[resp["generated_text"][-1] for resp in responses]

# Fine-tuning Pipeline
In the following we go through the steps required to setup a fine-tuning pipeline to impute missing survey data.

In [ ]:
# additional colab/kaggle setup
import sys
import os

def install_dependencies():
    import torch
    if not torch.cuda.is_available():
      print("CUDA is not available. \nPick a GPU before running this notebook. \nGo to 'Runtime' -> 'Change runtime type' to do this. (Colab)")
      return 
    %pip install transformers
    %pip install datasets
    %pip install peft
    %pip install evaluate
    %pip install wandb
    return



def is_running_in_kaggle():
    return 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

def is_running_in_colab():
    return "google.colab" in sys.modules

if is_running_in_colab() or is_running_in_kaggle():
    print("Running on Colab/Kaggle")
    install_dependencies()
else:
    print("Not running in Colab/Kaggle")

In [2]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, TaskType, prepare_model_for_kbit_training, get_peft_model
import torch
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict

seed = 24 # Please set your own favorite seed!
transformers.set_seed(seed)

## Data preperation
We use 2016 American National Election Studies survey data. Specifically, the subset of data Argyle et al. (2022) used in study 2 (https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/JPV20K).

In [ ]:
df_survey = pd.read_csv("data/2016_anes_argyle.csv")
df_survey

In [ ]:
# null values and data types
df_survey.info()

In [5]:
features = [
    "race",
    "discuss_politics",
    "ideology",
    "party",
    "church_goer",
    "age",
    "gender",
    "political_interest",
    "patriotism",
    "state",
]
label = "ground_truth" # this is the vote choice in this dataset

In [ ]:
# we tread missing values as a category 
df_survey_processed = (
    df_survey
    .astype({"age": str})
    .fillna("missing")
)
df_survey_processed

### Train/Test split

Any manipulation of the training data should be done in the step.

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df_survey_processed, test_size=0.2, random_state=seed)

# we can modify the training data here to do different experiments
# for example excluding republican voters
# leans_republican = df_train["party"].apply(lambda x: "Republican" in x)
# df_train = df_train[~leans_republican]

dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train, preserve_index=False),
    "test": Dataset.from_pandas(df_test, preserve_index=False),
})
dataset

### Prompt Design

We will use an instruction-tuned model and will therefore define an instruction prompt here. Having distinct `user` and `assistant` text. Where the `user` prompt includes all conditioning of the model and the `assistant` text is the expected answer.

In [ ]:
instruction = (
    "Please perform a classification task. "
    "Given the 2016 survey answers from the American National Election Studies, "
    "return which candidate the person voted for. "
    "Return a label from ['Trump', 'Clinton', 'Non-voter'] only without any other text.\n"
)
print(instruction)

In [ ]:
column_name_map = {
    "race": "Race",
    "discuss_politics": "Discusses politics",
    "ideology": "Ideology",
    "party": "Party",
    "church_goer": "Church",
    "age": "Age",
    "gender": "Gender",
    "political_interest": "Political interest",
    "patriotism": "American Flag",
    "state": "State",
    "ground_truth": "Vote",
}


def build_prompt_completion(
    row: dict,
    system_prompt: str = instruction,
) -> list[list[dict]]:
    user_prompt = "\n".join(
        [f"{column_name_map[k]}: {v}" for k, v in row.items() if k != label]
    )
    assistant_prompt = row[label]
    return {
        "prompt": [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
        "completion": [
            {
                "role": "assistant",
                "content": assistant_prompt,
            },
        ],
    }


build_prompt_completion(
    row=dataset["train"][0],
)

In [ ]:
dataset_llm = dataset.map(build_prompt_completion).remove_columns(features+[label])
dataset_llm

## Fine-tuning the model

### Model Selection

We use a very small (1B parameters) model here for demonstration proposes.

In [12]:
model_id = "unsloth/Llama-3.2-1B-Instruct"

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_id,
    # revision=revision, # NOTE: revision should be set for an reproducible experiment 
    trust_remote_code=True,
)

if getattr(tokenizer, "pad_token_id") is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

### Quantization
Quantization reduces the memory required to store the model (Dettmers et al., 2022). Typically a model is stored in 16-bit precision, therefore for a 70B parameter model:

$$\frac{16 \text{ bits}}{8 \text{ bits/byte}} \times 70 \times 10^9 \text{ parameters} = 140 \text{ GB of VRAM}$$

With 4-bit quantization, all parameters are stored in 4-bit precision, reducing the memory requirement to:

$$\frac{4 \text{ bits}}{8 \text{ bits/byte}} \times 70 \times 10^9 \text{ parameters} = 35 \text{ GB of VRAM}$$



In [ ]:
# load model in 4bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

### LoRA
Low-Rank Adapters (LoRA) are a parameter efficient fine-tuning method (Hu et al., 2021). Instead of finetuning all model weights, LoRA finetunes the weights of the adapter layers only. This requires less memory and allows for faster finetuning.

![LoRA Diagram](https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/peft/lora_diagram.png) 


In [ ]:
lora_rank = 8
lora_alpha = 8

lora_config = LoraConfig(
    r=lora_rank,
    lora_alpha=lora_alpha,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM,
    target_modules="all-linear",
)

## Training helper functions

In [ ]:
import trl

trl.__version__

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto",
)

if getattr(model.config, "pad_token_id") is None:
    model.config.pad_token_id = tokenizer.pad_token_id

In [ ]:
import wandb
from sklearn.metrics import accuracy_score

labels = df_survey_processed.ground_truth.unique()

first_token_ids = [
    tokenizer.encode(label, add_special_tokens=False)[0]
    for label in labels
]

def preprocess_logits_for_metrics(logits, labels):
    if isinstance(logits, tuple):
        # Depending on the model and config, logits may contain extra tensors,
        # like past_key_values, but logits always come first
        logits = logits[0]
    # Return dist of relevant tokens
    return logits[:, :, first_token_ids]

y_probas = []

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    # preds have the same shape as the labels, after the argmax(-1) has been calculated
    # by preprocess_logits_for_metrics but we need to shift the labels
    labels = labels[:, 1:]
    logits = logits[:, :-1]

    logits_first_token_dist = np.array(
        [
            [
                logit_id
                for logit_id, label_id in zip(logit_ids, label_ids)
                if label_id != -100
            ][0]
            for logit_ids, label_ids in zip(logits, labels)
        ]
    )

    logits_first_token_dist_exp = np.exp(logits_first_token_dist)
    normalized_logits_first_token_dist = (
        logits_first_token_dist_exp
        / logits_first_token_dist_exp.sum(axis=1, keepdims=True)
    )

    y_probas.append(normalized_logits_first_token_dist)

    df_y_proba = pd.DataFrame(
        normalized_logits_first_token_dist, columns=labels
    )

    wandb.log(
        data={
            "y_proba": wandb.Table(data=df_y_proba),
        }
    )

    # -100 is a default value for ignore_index used by DataCollatorForCompletionOnlyLM
    mask = labels == -100
    labels[mask] = tokenizer.pad_token_id

    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    decoded_preds = list(df_y_proba.idxmax(axis=1))

    return {
        "accuracy": accuracy_score(y_pred=decoded_preds, y_true=decoded_labels)
    }

## Training the model
To run the training without wandb logging, set `wandb.init(mode='disabled')`.

In [ ]:
from trl import SFTConfig, SFTTrainer

training_args = SFTConfig(
    # batch size
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # gradient_accumulation_steps=gradient_accumulation_steps,
    # completion_only_loss=True,
    # eval
    do_eval=True,
    eval_strategy="epoch",
    # saving
    save_strategy="epoch",
    save_total_limit=1,
    # other
    # max_seq_length=max_seq_length,
    output_dir="./results",
    # num_train_epochs=self.n_epochs,
)

trainer = SFTTrainer(
    tokenizer=tokenizer,
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    args=training_args,
    peft_config=lora_config,
    compute_metrics=compute_metrics,
    # data_collator=collator,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

trainer.evaluate()  # for zero-shot evaluation
trainer.train()

trainer.evaluate()
trainer.train()
trainer.evaluate()

wandb.finish()

All fine-tuning results can be found live at: https://wandb.ai/tobihol/survai-finetuning